In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Khởi tạo WebDriver
driver = webdriver.Chrome()

# Mở trang YouTube Music của kênh
main_url = 'https://music.youtube.com/channel/UCTTG37kYHtXBf9skWGQkjsw'
driver.get(main_url)

# Đảm bảo trang đã tải đầy đủ
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
time.sleep(3)

# Danh sách lưu dữ liệu bài hát
data = []
album_map = {}  # Lưu ánh xạ list_id -> album_name

# --- PHẦN 1: Lấy bài hát từ trang chính của kênh ---
try:
    show_all_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//button[.//span[contains(text(), "Show all")]]'))
    )
    ActionChains(driver).move_to_element(show_all_button).click().perform()
    print("Đã click 'Show all'")
    time.sleep(3)
except Exception as e:
    print("Không tìm thấy hoặc không thể click nút 'Show all':", e)

# Tìm tất cả liên kết bài hát ở trang chính
song_links = driver.find_elements(By.XPATH, '//a[@class="yt-simple-endpoint style-scope yt-formatted-string"]')
for song in song_links:
    song_url = song.get_attribute('href')
    song_title = song.text
    if song_url and 'watch?v=' in song_url:
        # Trích xuất list ID nếu có
        list_id = None
        if 'list=' in song_url:
            list_id = song_url.split('list=')[1].split('&')[0]
        album_name = song_title  # Mặc định nếu không tìm thấy album
        if list_id and list_id in album_map:
            album_name = album_map[list_id]
        data.append({
            "tracklist": song_title,  # Đổi từ 'title' thành 'tracklist'
            "video_url": song_url,
            "album": album_name
        })

# Quay lại trang chính để xử lý phần 2
driver.get(main_url)
time.sleep(3)

# --- PHẦN 2: Lấy bài hát từ các album (SINGLE & EPs) ---
try:
    more_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='More']"))
    )
    more_button.click()
    time.sleep(3)
except Exception as e:
    print("Không tìm thấy nút 'More' cho SINGLE & EPs:", e)

# Tìm các liên kết album và tên album
album_elements = driver.find_elements(By.XPATH, "//a[@class='yt-simple-endpoint image-wrapper style-scope ytmusic-two-row-item-renderer']")
album_titles = driver.find_elements(By.XPATH, "//yt-formatted-string[@class='title style-scope ytmusic-two-row-item-renderer']")

album_links = [el.get_attribute("href") for el in album_elements]
album_names = [el.text for el in album_titles]

# Lặp qua từng album
for idx, album_link in enumerate(album_links):
    album_name = album_names[idx] if idx < len(album_names) else "Unknown Album"
    driver.get(album_link)
    time.sleep(3)
    
    try:
        tracks = driver.find_elements(By.XPATH, "//yt-formatted-string[@class='title style-scope ytmusic-responsive-list-item-renderer complex-string']//a")
        for track in tracks:
            track_title = track.text  # Thay 'title' thành 'tracklist'
            href = track.get_attribute("href")
            if href and 'watch?v=' in href:
                list_id = None
                if 'list=' in href:
                    list_id = href.split('list=')[1].split('&')[0]
                    album_map[list_id] = album_name  # Gán ánh xạ album cho list_id
                data.append({
                    "album": album_name,
                    "tracklist": track_title,  # Đổi từ 'title' thành 'tracklist'
                    "video_url": href,
                })
    except Exception as e:
        print(f"Lỗi khi lấy bài hát từ album {album_link}: {e}")

# Đóng trình duyệt
driver.quit()

# Tạo DataFrame
df = pd.DataFrame(data)

# Lưu ra file Excel
df.to_excel("youtube_music_tracks.xlsx", index=False)



Đã click 'Show all'


In [152]:
df

,tracklist,video_url,album
0,Trước Khi Tuổi Trẻ Này Đóng Lối (feat. Dick & ...,https://music.youtube.com/watch?v=b_HPA2vw1SM&...,Trước Khi Tuổi Trẻ Này Đóng Lối (feat. Dick & ...
1,Sống Cho Hết Đời Thanh Xuân 4 (Chill Ver),https://music.youtube.com/watch?v=Mz_aYqv42po&...,Sống Cho Hết Đời Thanh Xuân 4 (Chill Ver)
2,Tình Yêu Là Thế,https://music.youtube.com/watch?v=ZMXqGutrXSk&...,Tình Yêu Là Thế
3,anh muốn mình như con thuyền kia lao vào em rồ...,https://music.youtube.com/watch?v=8ZMsmZHE5Qs&...,anh muốn mình như con thuyền kia lao vào em rồ...
4,Niềm tin nhầm chỗ,https://music.youtube.com/watch?v=bPvGgxtJJEo&...,Niềm tin nhầm chỗ
...,...,...,...
88,Đường Về,https://music.youtube.com/watch?v=lZyxeRdvRBk&...,Đường Về
89,Em Ơi Đành Thôi,https://music.youtube.com/watch?v=wfGVnXL2O0s&...,Em Ơi Đành Thôi
90,Merry x U (AM Remix),https://music.youtube.com/watch?v=gR-7bxPkxPc&...,Merry x U (AM Remix)
91,Em là em bé iu (Remix),https://music.youtube.com/watch?v=Flxmxl4bDxo&...,Em là em bé iu (Remix)
